# Geo Aggregator
BEA has discontinued publication of statistics for metropolitan statistical areas, micropolitan statistical areas, metropolitan divisions, combined statistical areas, and metropolitan and nonmetropolitan portions. GDP and personal income estimates will continue to be published by county.

We present here a way to derive, in some cases, approximations using the still available county estimates and national industry-level price estimates. These estimates will be of lesser quality than those produced internally at BEA for several reasons: the published industry price data is less detailed than is used internally, the public county data for current dollar GDP contains suppressions, we don't handle cases where non-standard price indexing fails, and using published data introduces rounding at an intermediate stage of calculation.

See the [README](https://github.com/us-bea/bea_geo_aggregator) for more information including links to methodology documentation.


## Usage

We assume you have completed steps 1-2 of the [README](https://github.com/us-bea/bea_geo_aggregator) (including getting a [BEA API key](https://www.bea.gov/API/signup/index.cfm)).

Running:
1. Put the BEA API key in-between the quotes in the `bea_key` variable below.
2. Define `start_year` and `end_year`. The earliest `start_year` is 2001 for GDP and 1969 for Personal Income.
3. For `geofips_filter` put the FIPS code for the Geographic aggregate (match the `geo_agg` type below). To see options, see [https://apps.bea.gov/regional/geography.htm].
2. For `geo_agg`, you can use `"MSA"` (Metropolitan Statistical Area) or `"PORT"` (Metropolitan and Nonmetropolitan portions). If you want to do your own mapping, see the `_full` version of this notebook.
3. Personal Income: Fill in `pi_tablename`. Options include `"CAINC1"`, `"CAINC4"`, `"CAINC5N"`, `"CAINC6N"`, and `"CAINC30"`.
4. Then click `Run All` above. You will have to click through a standard warning (“this code was not developed by Google”). It may ask your permission to download multiple files.
5. Results will be saved to files


Notes: 
- Values that cannot be computed will be shown as missing/NaN. For more explanation on why some values are missing, see the repository `README.md`.
- For GDP, we show results in two ways: geographic aggregate results separated by BEA table (CAGDP2/8/9/11), and an integrated table with all four geographic aggregate measures plus columns for constituent county current dollar GDP (this can be helpful to explain why some results can not be calculated). See the former formulation for table notes (they are not shown in the latter). For PI, we do the same, though there is only a single table in the first approach.
- Variable description: `GDP_current_dollars`="GDP in current dollars" (e.g., table CAGDP2), `chain_type_q_index`="Chained-type quantity index for real GDP" (e.g., table CAGDP8), `real_GDP_chained_dollars`="real GDP in chained dollars" (e.g., table CAGDP9), `contrib_pct_change`="Contributions to percent change in real GDP" (e.g., table CAGDP11).

In [1]:
# User configuraiton details
bea_key=""
start_year=2020
last_year=2024
geofips_filter = "10180" #Fips code for the Geographic Aggregate (e.g., "10180" is for "Abilene, TX (Metropolitan Statistical Area)")
geo_agg="MSA" # Aggregate type
pi_tablename = "CAINC4"

In [2]:
# Use key from .env file if available
try:
    from dotenv import dotenv_values
    if "beakey" in dotenv_values():
        bea_key = dotenv_values()["beakey"]
except ImportError:
    pass

In [3]:
import pandas as pd
# output variables
gdp_outfname="gdp_calculations.xlsx"
gdp_w_cnty_outfname="gdp_calculations_w_cnty.xlsx"
pi_outfname="pi_calculations.xlsx"
pi_w_cnty_outfname="pi_calculations_w_cnty.xlsx"

In [ ]:
import bea_geo_aggregator as REA_utils
fprefix = "" # allow to be run outside repo folder
print(f"Using tool version {REA_utils.__version__}")

## Calculate GDP statistics

In [5]:
# Calculate estimates
gdp_dfs = REA_utils.pull_merge_calc_gdp(start_year=start_year, last_year=last_year, geo_agg=geo_agg, bea_key=bea_key, geofips_filter=geofips_filter, verbosity=1)
combined_gdp_df = REA_utils.gdp_combine_cnty_nominals_geoagg_measures(gdp_dfs, geo_agg=geo_agg, bea_key=bea_key)

Pulling source data. Linecodes: 100%|██████████| 34/34 [00:07<00:00,  4.44it/s]


Calculating county-year non-missing data to use.


Pulling source data. Linecodes: 100%|██████████| 34/34 [00:06<00:00,  4.87it/s]


In [6]:
# save results to a file
REA_utils.REA_utils.dict_ret_to_excel(gdp_dfs, gdp_outfname)
combined_gdp_df.to_excel(gdp_w_cnty_outfname, index=False)

In [7]:
print("Show with counties. Just the top few rows")
REA_utils.display_with_notes(combined_gdp_df.head())

Show with counties. Just the top few rows


LineCode,Description,TimePeriod,price,Callahan GDP_current_dollars,Jones GDP_current_dollars,Taylor GDP_current_dollars,"Abilene, TX (Metropolitan Statistical Area) GDP_current_dollars","Abilene, TX (Metropolitan Statistical Area) chain_type_q_index","Abilene, TX (Metropolitan Statistical Area) real_GDP_chained_dollars","Abilene, TX (Metropolitan Statistical Area) contrib_pct_change"
1,All industry total,2020,105.398,315697,452040,7146068,7913805,109.612625,7483421,-0.027346
1,All industry total,2021,110.184,344214,538752,7647489,8530455,113.174179,7726573,3.249219
1,All industry total,2022,118.035,460386,661523,8525086,9646995,119.860320,8183046,5.907833
1,All industry total,2023,122.389,524585,609194,9237738,10371517,124.150433,8475939,3.579261
1,All industry total,2024,125.430,552729,599546,9865588,11017863,128.555087,8776651,3.547836


No notes.


In [8]:
# Show results w/o counties (and separate by table)
for tablename, df_agg in gdp_dfs.items():
    print(f"\nTable: {tablename} (just the top few rows)")
    REA_utils.display_with_notes(df_agg.head())


Table: CAGDP2 (just the top few rows)


GeoFips,GeoName,LineCode,Description,TimePeriod,DataValue,CL_UNIT,UNIT_MULT,Code
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2020,7913805,Thousands of dollars,3,CAGDP2-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2021,8530455,Thousands of dollars,3,CAGDP2-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2022,9646995,Thousands of dollars,3,CAGDP2-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2023,10371517,Thousands of dollars,3,CAGDP2-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2024,11017863,Thousands of dollars,3,CAGDP2-1


Notes:
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- (D): Not shown to avoid disclosure of confidential information; estimates are included in higher-level totals.
- 2: The private goods-producing industries consist of agriculture, forestry, fishing, and hunting; mining, quarrying, and oil and gas extraction; construction; and manufacturing.
- 3: The private services-producing industries consist of utilities; wholesale trade; retail trade; transportation and warehousing, excluding Postal Service; information; finance and insurance; real estate, rental, and leasing; professional, scientific, and technical services; management of companies; administrative and support and waste management and remediation services; educational services; health care and social assistance; arts, entertainment, and recreation; accommodation and food services; and other services (except government and government enterprises).

Table: CAGDP8 (just the top few

GeoFips,GeoName,LineCode,Description,TimePeriod,DataValue,CL_UNIT,UNIT_MULT,Code
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2020,109.612625,Quantity index,0,CAGDP8-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2021,113.174179,Quantity index,0,CAGDP8-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2022,119.860320,Quantity index,0,CAGDP8-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2023,124.150433,Quantity index,0,CAGDP8-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2024,128.555087,Quantity index,0,CAGDP8-1


Notes:
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- (D): Not shown to avoid disclosure of confidential information; estimates are included in higher-level totals.
- 2: The private goods-producing industries consist of agriculture, forestry, fishing, and hunting; mining, quarrying, and oil and gas extraction; construction; and manufacturing.
- 3: The private services-producing industries consist of utilities; wholesale trade; retail trade; transportation and warehousing, excluding Postal Service; information; finance and insurance; real estate, rental, and leasing; professional, scientific, and technical services; management of companies; administrative and support and waste management and remediation services; educational services; health care and social assistance; arts, entertainment, and recreation; accommodation and food services; and other services (except government and government enterprises).

Table: CAGDP9 (just the top few

...

Notes:
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- (D): Not shown to avoid disclosure of confidential information; estimates are included in higher-level totals.
- 2: The private goods-producing industries consist of agriculture, forestry, fishing, and hunting; mining, quarrying, and oil and gas extraction; construction; and manufacturing.
- 3: The private services-producing industries consist of utilities; wholesale trade; retail trade; transportation and warehousing, excluding Postal Service; information; finance and insurance; real estate, rental, and leasing; professional, scientific, and technical services; management of companies; administrative and support and waste management and remediation services; educational services; health care and social assistance; arts, entertainment, and recreation; accommodation and food services; and other services (except government and government enterprises).

Table: CAGDP11 (just the top fe

GeoFips,GeoName,LineCode,Description,TimePeriod,DataValue,CL_UNIT,UNIT_MULT,Code
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2020,-0.027346,Percent change,0,CAGDP11-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2021,3.249219,Percent change,0,CAGDP11-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2022,5.907833,Percent change,0,CAGDP11-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2023,3.579261,Percent change,0,CAGDP11-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2024,3.547836,Percent change,0,CAGDP11-1


Notes:
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- (D): Not shown to avoid disclosure of confidential information; estimates are included in higher-level totals.
- 2: The private goods-producing industries consist of agriculture, forestry, fishing, and hunting; mining, quarrying, and oil and gas extraction; construction; and manufacturing.
- 3: The private services-producing industries consist of utilities; wholesale trade; retail trade; transportation and warehousing, excluding Postal Service; information; finance and insurance; real estate, rental, and leasing; professional, scientific, and technical services; management of companies; administrative and support and waste management and remediation services; educational services; health care and social assistance; arts, entertainment, and recreation; accommodation and food services; and other services (except government and government enterprises).

Table: CAGDP1 (just the top few

GeoFips,GeoName,LineCode,Description,TimePeriod,DataValue,CL_UNIT,UNIT_MULT,Code
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2020,7483420.514245,Thousands of dollars,3,CAGDP1-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2021,7726573.218811,Thousands of dollars,3,CAGDP1-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2022,8183046.253735,Thousands of dollars,3,CAGDP1-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2023,8475938.802275,Thousands of dollars,3,CAGDP1-1
10180,"Abilene, TX (Metropolitan Statistical Area)",1,All industry total,2024,8776651.180001,Thousands of dollars,3,CAGDP1-1


Notes:
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- (D): Not shown to avoid disclosure of confidential information; estimates are included in higher-level totals.
- 2: The private goods-producing industries consist of agriculture, forestry, fishing, and hunting; mining, quarrying, and oil and gas extraction; construction; and manufacturing.
- 3: The private services-producing industries consist of utilities; wholesale trade; retail trade; transportation and warehousing, excluding Postal Service; information; finance and insurance; real estate, rental, and leasing; professional, scientific, and technical services; management of companies; administrative and support and waste management and remediation services; educational services; health care and social assistance; arts, entertainment, and recreation; accommodation and food services; and other services (except government and government enterprises).


## Calculate Personal Income statistics

In [9]:
# Do calculation
pi_df = REA_utils.pull_merge_calc_pi(tablename=pi_tablename, start_year=start_year, last_year=last_year, 
                                   geo_agg=geo_agg, bea_key=bea_key, geofips_filter=geofips_filter, verbosity=1)
combined_pi_df = REA_utils.pi_combine_cnty_geoagg_calc(pi_df, geo_agg=geo_agg, bea_key=bea_key)

Pulling source data. Linecodes: 100%|██████████| 20/20 [00:05<00:00,  3.54it/s]


Main calculations


Pulling source data. Linecodes: 100%|██████████| 20/20 [00:29<00:00,  1.49s/it]


In [10]:
# Save results to a file
pi_df.to_excel(pi_outfname, sheet_name=pi_tablename, index=False)
combined_pi_df.to_excel(pi_w_cnty_outfname, sheet_name=pi_tablename, index=False)

In [11]:
print("Show with counties. Just the top few rows")
REA_utils.display_with_notes(combined_pi_df.head())

Show with counties. Just the top few rows


LineCode,Description,TimePeriod,Callahan DataValue,Jones DataValue,Taylor DataValue,Code,"Abilene, TX (Metropolitan Statistical Area) DataValue",CL_UNIT,UNIT_MULT
10,Personal income,2020,672831,697110,7100349,CAINC4-10,8470290,Thousands of dollars,3
10,Personal income,2021,715193,771061,7786490,CAINC4-10,9272744,Thousands of dollars,3
10,Personal income,2022,743589,856433,8182619,CAINC4-10,9782641,Thousands of dollars,3
10,Personal income,2023,790322,852897,8700109,CAINC4-10,10343328,Thousands of dollars,3
10,Personal income,2024,822731,854914,9180520,CAINC4-10,10858165,Thousands of dollars,3


No notes.


In [12]:
# Show results w/o counties
REA_utils.display_with_notes(pi_df.head())

GeoFips,GeoName,LineCode,Description,TimePeriod,Code,DataValue,CL_UNIT,UNIT_MULT
10180,"Abilene, TX (Metropolitan Statistical Area)",10,Personal income,2020,CAINC4-10,8470290,Thousands of dollars,3
10180,"Abilene, TX (Metropolitan Statistical Area)",10,Personal income,2021,CAINC4-10,9272744,Thousands of dollars,3
10180,"Abilene, TX (Metropolitan Statistical Area)",10,Personal income,2022,CAINC4-10,9782641,Thousands of dollars,3
10180,"Abilene, TX (Metropolitan Statistical Area)",10,Personal income,2023,CAINC4-10,10343328,Thousands of dollars,3
10180,"Abilene, TX (Metropolitan Statistical Area)",10,Personal income,2024,CAINC4-10,10858165,Thousands of dollars,3


Notes:
- Note.: All dollar estimates are in thousands of current dollars (not adjusted for inflation). Calculations are performed on unrounded data.
-  : Last updated: February 5, 2026-- new statistics for 2024; revised statistics for 2020-2023.
- 1: Nonfarm personal income is total personal income less farm income.
- 2: Farm income is farm earnings less farm employer contributions for social insurance.
- 3: U.S. Census Bureau midyear population estimates.
- 4: Per capita personal income is total personal income divided by total midyear population.
- 5: Employer contributions for government social insurance are included in earnings by industry and earnings by place of work, but they are excluded from net earnings by place of residence and personal income. Employee and self-employed contributions are subtractions in the calculation of net earnings by place of residence and all of the income measures.
- 6: The adjustment for residence is the net inflow of the earnings of interarea commut